In [2]:
import torch.nn as nn
import torch

In [10]:
class ConsecutiveDifferenceLossAndBatch(nn.Module):
    def __init__(self, consecutive_size):
        super(ConsecutiveDifferenceLossAndBatch, self).__init__()
        self.consecutive_size = consecutive_size

    def forward(self, prediction, target):
        pred_reshape = prediction.view(-1, self.consecutive_size)
        target_reshape = target.view(-1, self.consecutive_size)
        pred_dif = pred_reshape[:, 1:] - pred_reshape[:, :-1]
        target_dif = target_reshape[:, 1:] - target_reshape[:, :-1]
        
        pred_dif_batch = pred_reshape[1:, :] - pred_reshape[:-1, :]
        target_dif_batch = target_reshape[1:, :] - target_reshape[:-1, :]
        
        consec_loss = torch.mean((pred_dif - target_dif) ** 2)
        batch_loss = torch.mean((pred_dif_batch - target_dif_batch) ** 2)
        
        return consec_loss + batch_loss

In [30]:
class ConsecutiveSlopeWeightedMSELoss(nn.Module):
    def __init__(self, consecutive_size):
        super(ConsecutiveSlopeWeightedMSELoss, self).__init__()
        """this is just mse weighted by how different the slope before and after the point is"""
        self.consecutive_size = consecutive_size

    def forward(self, prediction, target):
        pred_reshape = prediction.view(-1, self.consecutive_size)
        target_reshape = target.view(-1, self.consecutive_size)
        #pred_dif = pred_reshape[:, 1:] - pred_reshape[:, :-1]
        target_slope = target_reshape[:, 1:] - target_reshape[:, :-1]
        
        target_slope_dif = target_slope[:, 1:] - target_slope[:, :-1]
        
        
        errors = pred_reshape[:,1:-1] - target_reshape[:,1:-1]
        
        return torch.mean(target_slope_dif **2 * errors **2)
    

In [31]:
batch_size = 4
consecutive_size = 5

# Randomly generated data to simulate predictions and targets
prediction = torch.randn(batch_size, consecutive_size)
target = torch.randn(batch_size, consecutive_size)

# Instantiate the loss function
loss_fn = ConsecutiveSlopeWeightedMSELoss(consecutive_size)

# Calculate the loss
loss = loss_fn(prediction, target)
loss

tensor(19.3308)

In [9]:
prediction,target

(tensor([[ 0.5290,  0.2795, -0.0702,  0.5486, -1.1225],
         [-0.3446,  1.5929, -0.1428,  0.4850, -0.2112],
         [-0.4401, -0.2580, -0.4857, -0.9856, -1.0039],
         [ 0.8572, -0.7022,  2.3131,  0.0054,  0.9092]]),
 tensor([[-0.6925, -0.7514,  0.0041, -2.1649, -2.2122],
         [ 0.5277, -0.0382, -0.7986, -0.0239, -0.0115],
         [-0.6821, -0.3912,  0.0806, -0.6225,  0.0597],
         [ 0.2659, -0.5664, -0.4192, -0.9368,  0.0898]]))